In [1]:
from main import compute_urban_morphometrics

In [4]:
metrics = compute_urban_morphometrics(
    8.410025437655117,
    49.00199860395597,
    "https://download.geofabrik.de/europe/germany/baden-wuerttemberg/karlsruhe-regbez-latest.osm.pbf",
    return_dict=True,
    save_data_path="/mnt/c/Users/isaac/Downloads"
)

Using cached PBF: karlsruhe-regbez-latest.osm.pbf
Converting PBF to geodataframes...


2026-02-07 15:23:53,740 - metrics.compute - INFO - Computing metric: courtyard_area
2026-02-07 15:23:53,913 - metrics.compute - INFO - Finished metric: courtyard_area (0.17s)
2026-02-07 15:23:53,916 - metrics.compute - INFO - Computing metric: floor_area


Saved buildings, networks, and landuse to /mnt/c/Users/isaac/Downloads


2026-02-07 15:23:54,073 - metrics.compute - INFO - Finished metric: floor_area (0.16s)
2026-02-07 15:23:54,074 - metrics.compute - INFO - Computing metric: longest_axis_length
2026-02-07 15:23:54,253 - metrics.compute - INFO - Finished metric: longest_axis_length (0.18s)
2026-02-07 15:23:54,255 - metrics.compute - INFO - Computing metric: perimeter_wall
2026-02-07 15:23:54,562 - metrics.compute - INFO - Finished metric: perimeter_wall (0.31s)
2026-02-07 15:23:54,563 - metrics.compute - INFO - Computing metric: volume
2026-02-07 15:23:54,725 - metrics.compute - INFO - Finished metric: volume (0.16s)
2026-02-07 15:23:54,726 - metrics.compute - INFO - Computing metric: circular_compactness
2026-02-07 15:23:54,902 - metrics.compute - INFO - Finished metric: circular_compactness (0.18s)
2026-02-07 15:23:54,904 - metrics.compute - INFO - Computing metric: square_compactness
2026-02-07 15:23:55,060 - metrics.compute - INFO - Finished metric: square_compactness (0.15s)
2026-02-07 15:23:55,061 

Computed 54 metrics


In [5]:
for k in metrics.keys():
    print(k)

courtyard_area
floor_area
longest_axis_length
perimeter_wall
volume
circular_compactness
square_compactness
convexity
courtyard_index
rectangularity
shape_index
corners
squareness
equivalent_rectangular_index
elongation
facade_ratio
fractal_dimension
form_factor
compactness_weighted_axis
centroid_corner_distance
orientation
shared_walls
alignment
neighbor_distance
mean_interbuilding_distance
building_adjacency
neighbors
cell_alignment
street_alignment
courtyards
street_profile
nearest_street_distance
degree_vehicle
meshedness_vehicle
gamma_vehicle
edge_node_ratio_vehicle
cyclomatic_vehicle
meshedness_global_vehicle
gamma_global_vehicle
cds_length_total_vehicle
proportion_three_vehicle
proportion_four_vehicle
proportion_dead_vehicle
degree_pedestrian
meshedness_pedestrian
gamma_pedestrian
edge_node_ratio_pedestrian
cyclomatic_pedestrian
meshedness_global_pedestrian
gamma_global_pedestrian
cds_length_total_pedestrian
proportion_three_pedestrian
proportion_four_pedestrian
proportion_dead_

In [13]:
metrics["degree_pedestrian"]

,geometry,degree_pedestrian_mean,degree_pedestrian_std,degree_pedestrian_median,degree_pedestrian_sum,degree_pedestrian_p10,degree_pedestrian_p20,degree_pedestrian_p30,degree_pedestrian_p40,degree_pedestrian_p50,degree_pedestrian_p60,degree_pedestrian_p70,degree_pedestrian_p80,degree_pedestrian_p90
0,"POLYGON ((8.40608 49.00183, 8.40996 49.00313, ...",2.043825,1.043022,2.0,1026,1.0,1.0,1.0,1.0,2.0,3.0,3.0,3.0,3.0
